In [6]:
# script to combine summary urban form stats for all cities
# last edit Oct 25 2023 Peter Berrill

# load required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from shapely.wkt import loads,dumps
from shapely import speedups
from shapely.geometry import Point, LineString, Polygon
speedups.enable()
from pyproj import CRS
from pysal.lib import weights
import pickle
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from citymob import import_csv_w_wkt_to_gdf
import networkx as nx
import osmnx as ox
import seaborn as sns
from sklearn.metrics import r2_score
import matplotlib.ticker as mticker

crs0=3035

cities=['Berlin','Dresden','Düsseldorf','Frankfurt am Main','Kassel','Leipzig','Magdeburg','Potsdam','Clermont','Dijon','Lille','Lyon','Montpellier','Nantes','Nimes','Paris','Toulouse','Madrid','Wien']
countries=['Germany','Germany','Germany','Germany','Germany','Germany','Germany','Germany','France','France','France','France','France','France','France','France','France','Spain','Austria']
ua_year=['2018','2018','2018','2018','2018','2018','2018','2018','2012','2018','2018','2018','2012','2018','2018','2012','2012','2018','2012']
ua_ver=['v013','v013','v013','v013','v013','v013','v013','v013','revised_v021','v013','v013','v013','revised_v021','v013','v013','revised_v021','revised_v021','v013','revised_v021']

metrics=['n','m','k_avg','intersection_density_km','clean_intersection_density_km','street_length_total','street_density_km','streets_per_node_avg','street_length_avg']
nw_type='drive'
bld_dens_city=[]
pop_dens_city=[]
pop_city=[]
area_city=[]
d2c=[]
d2sc=[]
urb_fab=[]
urb=[]
comm=[]
int_dens=[]
street_len=[]
bike_share=[]
transit_access=[]

In [36]:
def get_area_shares(city):

    print(city)
    country=countries[cities.index(city)]

    # read in file of city boundaries
    if country=='Germany':
        fp='../outputs/city_boundaries/' + city + '_basic.csv'
    else: 
        fp='../outputs/city_boundaries/' + city + '.csv'
    gdf_boundary = import_csv_w_wkt_to_gdf(fp,crs=crs0,geometry_col='geometry')

    # land use
    year=ua_year[cities.index(city)]
    ver=ua_ver[cities.index(city)]

    if city=='Potsdam':
        fp='../../MSCA_data/UrbanAtlas/Berlin/Data/Berlin_UA2018_v013.gpkg'

    else: 
        fp='../../MSCA_data/UrbanAtlas/' + city + '/Data/' + city + '_UA' + year + '_' + ver +'.gpkg'
    LU=gpd.read_file(fp)
    LU.to_crs(crs0,inplace=True)

    LU_gdf=gpd.overlay(gdf_boundary,LU,how='intersection')
    LU_gdf['area2']=LU_gdf.area

    if year =='2012': LU_gdf.rename(columns={'class_2012':'class_2018'},inplace=True)
    # create urban fabric (residential) and commercial and road land-uses
    LU_gdf['Class']='Other'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Discontinuous dense urban fabric (S.L. : 50% -  80%)','Discontinuous medium density urban fabric (S.L. : 30% - 50%)','Discontinuous low density urban fabric (S.L. : 10% - 30%)','Discontinuous very low density urban fabric (S.L. : < 10%)','Continuous urban fabric (S.L. : > 80%)']),'Class']='Urban_Fabric'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Industrial, commercial, public, military and private units','Construction sites','Airports','Port areas']),'Class']='Industrial_Commercial'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Fast transit roads and associated land','Other roads and associated land']),'Class']='Road'

    # make a non-road classification, to calculate by reverse the road area
    LU_gdf['RoadStatus']='NonRoad'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Fast transit roads and associated land','Other roads and associated land']),'RoadStatus']='Road'

    # make an aggregated LU classification, this time distinguishing 'urban' and 'non-urban' areas
    LU_gdf['UrbanStatus']='Non-Urban'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Discontinuous dense urban fabric (S.L. : 50% -  80%)',
    'Discontinuous medium density urban fabric (S.L. : 30% - 50%)',
    'Discontinuous low density urban fabric (S.L. : 10% - 30%)',
    'Discontinuous very low density urban fabric (S.L. : < 10%)',
    'Continuous urban fabric (S.L. : > 80%)',
    'Industrial, commercial, public, military and private units',
    'Sports and leisure facilities',
    'Green urban areas',
    'Isolated structures',
    'Mineral extraction and dump sites', # should this be included?
    'Fast transit roads and associated land',
    'Railways and associated land ',
    'Airports',
    'Other roads and associated land',
    'Port areas',
    'Construction sites',
    ]),'UrbanStatus']='Urban'

    class_area=pd.DataFrame(LU_gdf.groupby('Class')['area2'].sum()/LU_gdf['area2'].sum()).reset_index()
    urb_fab.append(class_area.loc[class_area['Class']=='Urban_Fabric','area2'].values[0])
    comm.append(class_area.loc[class_area['Class']=='Industrial_Commercial','area2'].values[0])

    urb_area=pd.DataFrame(LU_gdf.groupby('UrbanStatus')['area2'].sum()/LU_gdf['area2'].sum()).reset_index() 
    urb.append(urb_area.loc[urb_area['UrbanStatus']=='Urban','area2'].values[0])


In [ ]:
c=pd.Series(cities)
c.apply(get_area_shares)

In [15]:
pdens_mix_all_DE=pd.DataFrame(columns = ['geocode','Population','City'])
for city in ['Dresden','Düsseldorf','Frankfurt am Main','Kassel','Leipzig','Magdeburg','Potsdam']:
    popdens_mix=pd.read_csv('../outputs/density_geounits/' + city + '_pop_density.csv',dtype={'geocode':str})
    popdens_mix['Population']=popdens_mix['Pop_2018']
    popdens_mix=popdens_mix.loc[:,['geocode','Population']]
    popdens_mix['City']=city
    pdens_mix_all_DE=pd.concat([pdens_mix_all_DE,popdens_mix])
pdens_mix_all_DE.reset_index(drop=True,inplace=True)
pdens_mix_all_DE['Country']='Germany'
pdens_mix_all_DE

,geocode,Population,City,Country
0,01067,15666.0,Dresden,Germany
1,01069,30154.0,Dresden,Germany
2,01097,17060.0,Dresden,Germany
3,01099,31714.0,Dresden,Germany
4,01108,6119.0,Dresden,Germany
...,...,...,...,...
169,14473,13451.0,Potsdam,Germany
170,14476,16667.0,Potsdam,Germany
171,14478,26578.0,Potsdam,Germany
172,14480,28694.0,Potsdam,Germany


In [16]:
pdens_mix_all_FR=pd.DataFrame(columns = ['geocode','Population','City','Country'])
for city in ['Clermont','Dijon','Lille','Lyon','Montpellier','Nantes','Nimes','Toulouse']:
    popdens_mix=pd.read_csv('../outputs/density_geounits/' + city + '_pop_density_mixres.csv',dtype={'geocode':str})
    popdens_mix=popdens_mix.loc[:,['geocode','Population']]
    popdens_mix['City']=city
    popdens_mix['Country']='France'
    pdens_mix_all_FR=pd.concat([pdens_mix_all_FR,popdens_mix])
pdens_mix_all_FR.reset_index(drop=True,inplace=True)
pdens_mix_all_FR

,geocode,Population,City,Country
0,101,10226.67737,Clermont,France
1,102,16336.893857,Clermont,France
2,103,25629.005422,Clermont,France
3,104,8984.275272,Clermont,France
4,105001,3015.902235,Clermont,France
...,...,...,...,...
909,055017,3758.383461,Toulouse,France
910,056001,1892.703835,Toulouse,France
911,056002,3982.885656,Toulouse,France
912,056003,1253.629968,Toulouse,France


In [18]:
d2_all=pd.DataFrame(columns = ['geocode','minDist_subcenter','Distance2Center','City','Country'])
for city in ['Clermont','Dijon','Lille','Lyon','Montpellier','Nantes','Nimes','Toulouse','Dresden','Düsseldorf','Frankfurt am Main','Kassel','Leipzig','Magdeburg','Potsdam']:
    country=countries[cities.index(city)]
    d2=pd.read_csv('../outputs/CenterSubcenter/' + city + '_dist.csv',dtype={'geocode':str})
    d2=d2.loc[:,['geocode','minDist_subcenter','Distance2Center']]
    d2['City']=city
    d2['Country']=country
    d2_all=pd.concat([d2_all,d2])
d2_all.reset_index(drop=True,inplace=True)
d2_all

,geocode,minDist_subcenter,Distance2Center,City,Country
0,101,1.830664,0.235196,Clermont,France
1,102,1.907124,0.536975,Clermont,France
2,103,0.305305,1.772067,Clermont,France
3,104,1.458591,0.647796,Clermont,France
4,105001,1.288424,1.758292,Clermont,France
...,...,...,...,...,...
1107,14467,1.72844,0.971364,Potsdam,Germany
1108,14469,1.190694,2.242515,Potsdam,Germany
1109,14476,6.128019,7.801996,Potsdam,Germany
1110,14473,3.265259,1.655724,Potsdam,Germany


In [13]:
trans_all=pd.DataFrame(columns = ['geocode','score_spatiotemporal_min','City','Country'])
for city in ['Clermont','Dijon','Lille','Lyon','Montpellier','Nantes','Nimes','Toulouse','Dresden','Düsseldorf','Frankfurt am Main','Kassel','Leipzig','Magdeburg','Potsdam']:
    country=countries[cities.index(city)]
    fp='../outputs/transit_access/'+city+'.csv'
    if country=='France':
        fp='../outputs/transit_access/'+city+'.csv'
    tr=pd.read_csv(fp,dtype={'geocode':str})
    tr['City']=city
    tr['Country']=country
    trans_all=pd.concat([trans_all,tr])
trans_all.reset_index(drop=True,inplace=True)
trans_all

,geocode,score_spatiotemporal_min,City,Country
0,101,15.920526,Clermont,France
1,102,12.134095,Clermont,France
2,103,9.963807,Clermont,France
3,104,15.252246,Clermont,France
4,105001,4.516913,Clermont,France
...,...,...,...,...
1087,14473,6.693432,Potsdam,Germany
1088,14476,1.312834,Potsdam,Germany
1089,14478,9.858654,Potsdam,Germany
1090,14480,11.258198,Potsdam,Germany


In [19]:
comb_FR=pdens_mix_all_FR.merge(d2_all,on=['Country','City','geocode'])
comb_FR=comb_FR.merge(trans_all,on=['Country','City','geocode'],how='left')

In [20]:
d2c_FR_other=np.average(comb_FR['Distance2Center'], weights=comb_FR['Population'])
d2sc_FR_other=np.average(comb_FR['minDist_subcenter'], weights=comb_FR['Population'])
d2c_FR_other

6.001820287433744

In [21]:
d2sc_FR_other

2.8758929938235887

In [22]:
comb_FR=comb_FR.fillna(0.0)

In [24]:
trans_FR_other=np.average(comb_FR['score_spatiotemporal_min'], weights=comb_FR['Population'])
trans_FR_other

11.387464486730796

In [25]:
comb_DE=pdens_mix_all_DE.merge(d2_all,on=['Country','City','geocode'])
comb_DE=comb_DE.merge(trans_all,on=['Country','City','geocode'],how='left')
comb_DE

,geocode,Population,City,Country,minDist_subcenter,Distance2Center,score_spatiotemporal_min
0,01067,15666.0,Dresden,Germany,1.625958,1.101689,59.819808
1,01069,30154.0,Dresden,Germany,2.54082,0.790464,78.41115
2,01097,17060.0,Dresden,Germany,0.37598,2.368241,67.834102
3,01099,31714.0,Dresden,Germany,2.104873,4.329216,6.116445
4,01108,6119.0,Dresden,Germany,3.493979,12.514477,2.534147
...,...,...,...,...,...,...,...
169,14473,13451.0,Potsdam,Germany,3.265259,1.655724,6.693432
170,14476,16667.0,Potsdam,Germany,6.128019,7.801996,1.312834
171,14478,26578.0,Potsdam,Germany,2.426146,4.891427,9.858654
172,14480,28694.0,Potsdam,Germany,0.522689,6.366003,11.258198


In [26]:
d2c_DE_other=np.average(comb_DE['Distance2Center'], weights=comb_DE['Population'])
d2sc_DE_other=np.average(comb_DE['minDist_subcenter'], weights=comb_DE['Population'])
d2c_DE_other

4.433429643903551

In [27]:
d2sc_DE_other

2.431991952045285

In [28]:
comb_DE=comb_DE.fillna(0.0)

In [29]:
trans_DE_other=np.average(comb_DE['score_spatiotemporal_min'], weights=comb_DE['Population'])
trans_DE_other

24.71747832666632

In [2]:
def get_UF_avg(city):
    print(city)
    country=countries[cities.index(city)]

    # bldg dens
    if city == 'Frankfurt am Main': 
        fp='../../MSCA_data/BuildingsDatabase/clips/eubucco_frankfurt_am_main.shp'
    else:
        fp='../../MSCA_data/BuildingsDatabase/clips/eubucco_' + city + '.shp'
    buildings_gdf=gpd.read_file(fp)

    # read in file of city boundaries
    if country=='Germany':
        fp='../outputs/city_boundaries/' + city + '_basic.csv'
    else: 
        fp='../outputs/city_boundaries/' + city + '.csv'
    gdf_boundary = import_csv_w_wkt_to_gdf(fp,crs=crs0,geometry_col='geometry')

    buildings_gdf=gpd.sjoin(buildings_gdf,gdf_boundary)

    # calculate the area of the buidlings from the database
    buildings_gdf["area"] = buildings_gdf["geometry"].area
    # calculate the centerpoint of each building geometry, 
    buildings_gdf["center"] = buildings_gdf["geometry"].centroid

    buildings_gdf['volume']=buildings_gdf['area']*buildings_gdf['height']
    bld_dens=buildings_gdf['volume'].sum()/gdf_boundary.area
    bld_dens_city.append(bld_dens.values[0])

    # pop dens
    dens=pd.read_excel('../outputs/density_geounits/summary_stats_' + city + '.xlsx',sheet_name='area_pop_sum')
    country=countries[cities.index(city)]
    # pop density city wide
    if country=='Germany':
        pop_dens=dens.loc[dens['variable']=='density','value'].reset_index(drop=True)[0]
        pop_total=dens.loc[dens['variable']=='Pop_2018','value'].reset_index(drop=True)[0]
        area=dens.loc[dens['variable']=='Area','value'].reset_index(drop=True)[0]
    else:
        pop_dens=dens.loc[dens['index']=='density',0].values[0]
        pop_total=dens.loc[dens['index']=='population',0].values[0]
        area=dens.loc[dens['index']=='area',0].values[0]   

    pop_dens_city.append(pop_dens)
    pop_city.append(pop_total)
    area_city.append(area)

    # pop density local, used for weighting distance and transit access metrics
    if country=='Germany':
        pop_dens_local=pd.read_csv('../outputs/density_geounits/' + city + '_pop_density.csv')
        pop_dens_local['Population']=pop_dens_local['Pop_2018']
    elif (country in ['France', 'Spain']) & (city != 'Paris'):
        pop_dens_local=pd.read_csv('../outputs/density_geounits/' + city + '_pop_density_mixres.csv')
        pop_dens_local['Population']=pop_dens_local['area']*pop_dens_local['Density']
    elif city =='Paris':
        pop_dens_local=pd.read_csv('../outputs/density_geounits/' + city + '_pop_density_lowres.csv')
        pop_dens_local.rename(columns={'geo_unit':'geocode'},inplace=True)
    else:
        pop_dens_local=pd.read_csv('../outputs/density_geounits/' + city + '_pop_density.csv')

    # distances, weighted avg by population
    data=pd.read_csv('../outputs/Combined/'+city+'_UF.csv')
    data=data.loc[:,['Res_geocode','HHNR','DistSubcenter_res', 'DistCenter_res']].drop_duplicates()

    data=pd.merge(data,pop_dens_local.loc[:,['geocode','Population']],left_on='Res_geocode',right_on='geocode')
    data=data.fillna(value=0.0) 
    dc=np.average(data['DistCenter_res'],weights=data['Population'])
    dsc=np.average(data['DistSubcenter_res'],weights=data['Population'])

    d2c.append(dc)
    d2sc.append(dsc)

    # transit access
    fp='../outputs/transit_access/'+city+'.csv'
    tr=pd.read_csv(fp)
    # trmean=tr['score_spatiotemporal'].mean()
    tr_weighted=pd.merge(tr,pop_dens_local.loc[:,['geocode','Population']])
    tr_weighted=tr_weighted.fillna(value=0.0) 
    trmean=np.average(tr_weighted['score_spatiotemporal'], weights=tr_weighted['Population'])
    transit_access.append(trmean)

    # land use
    year=ua_year[cities.index(city)]
    ver=ua_ver[cities.index(city)]

    if city=='Potsdam':
        fp='../../MSCA_data/UrbanAtlas/Berlin/Data/Berlin_UA2018_v013.gpkg'

    else: 
        fp='../../MSCA_data/UrbanAtlas/' + city + '/Data/' + city + '_UA' + year + '_' + ver +'.gpkg'
    LU=gpd.read_file(fp)
    LU.to_crs(crs0,inplace=True)

    LU_gdf=gpd.overlay(gdf_boundary,LU,how='intersection')

    if year =='2012': LU_gdf.rename(columns={'class_2012':'class_2018'},inplace=True)
    # create urban fabric (residential) and commercial and road land-uses
    LU_gdf['Class']='Other'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Discontinuous dense urban fabric (S.L. : 50% -  80%)','Discontinuous medium density urban fabric (S.L. : 30% - 50%)','Discontinuous low density urban fabric (S.L. : 10% - 30%)','Discontinuous very low density urban fabric (S.L. : < 10%)','Continuous urban fabric (S.L. : > 80%)']),'Class']='Urban_Fabric'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Industrial, commercial, public, military and private units','Construction sites','Airports','Port areas']),'Class']='Industrial_Commercial'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Fast transit roads and associated land','Other roads and associated land']),'Class']='Road'

    # make a non-road classification, to calculate by reverse the road area
    LU_gdf['RoadStatus']='NonRoad'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Fast transit roads and associated land','Other roads and associated land']),'RoadStatus']='Road'

    # make an aggregated LU classification, this time distinguishing 'urban' and 'non-urban' areas
    LU_gdf['UrbanStatus']='Non-Urban'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Discontinuous dense urban fabric (S.L. : 50% -  80%)',
    'Discontinuous medium density urban fabric (S.L. : 30% - 50%)',
    'Discontinuous low density urban fabric (S.L. : 10% - 30%)',
    'Discontinuous very low density urban fabric (S.L. : < 10%)',
    'Continuous urban fabric (S.L. : > 80%)',
    'Industrial, commercial, public, military and private units',
    'Sports and leisure facilities',
    'Green urban areas',
    'Isolated structures',
    'Mineral extraction and dump sites', # should this be included?
    'Fast transit roads and associated land',
    'Railways and associated land ',
    'Airports',
    'Other roads and associated land',
    'Port areas',
    'Construction sites',
    ]),'UrbanStatus']='Urban'

    class_area=pd.DataFrame(LU_gdf.groupby('Class')['area'].sum()/LU_gdf['area'].sum()).reset_index()
    urb_fab.append(class_area.loc[class_area['Class']=='Urban_Fabric','area'].values[0])
    comm.append(class_area.loc[class_area['Class']=='Industrial_Commercial','area'].values[0])

    urb_area=pd.DataFrame(LU_gdf.groupby('UrbanStatus')['area'].sum()/LU_gdf['area'].sum()).reset_index()

    # connectivity

    poly = gdf_boundary.to_crs(4326).iloc[0].geometry

    graph_plz=ox.graph_from_polygon(poly,simplify=True,network_type=nw_type,retain_all=True)
    graph_proj=ox.project_graph(graph_plz)

    # get area
    graph_area=gdf_boundary.area.values[0]

    # get basic stats
    stats = ox.basic_stats(graph_proj,area=graph_area,clean_int_tol=10)

    # restrict to stats we are interested in
    stats1=dict((k, stats[k]) for k in metrics if k in stats)
    op=pd.Series(stats1)

    cf = '["cycleway"]'
    # this might throw an error if no graph is found within the polygon
    graph_bike = ox.graph_from_polygon(poly, custom_filter=cf,retain_all=True)
    if len(graph_bike.edges)>0:
        graph_proj2=ox.project_graph(graph_bike)
        stats = ox.basic_stats(graph_proj2,area=graph_area,clean_int_tol=10)
        stats2=dict((k, stats[k]) for k in metrics if k in stats)
        bike_lane_share=round(stats2['street_length_total']/stats1['street_length_total'],4)
    else:
        bike_lane_share=0
    #op=op.append(pd.Series({'bike_lane_share': bike_lane_share}))

    int_dens.append(op['clean_intersection_density_km'])
    street_len.append(op['street_length_avg'])
    bike_share.append(bike_lane_share)

In [7]:
def get_UF_avg_lite(city):
    print(city)
    country=countries[cities.index(city)]

    # pop dens
    dens=pd.read_excel('../outputs/density_geounits/summary_stats_' + city + '.xlsx',sheet_name='area_pop_sum')
    country=countries[cities.index(city)]
    # pop density city wide
    if country=='Germany':
        pop_dens=dens.loc[dens['variable']=='density','value'].reset_index(drop=True)[0]
        pop_total=dens.loc[dens['variable']=='Pop_2018','value'].reset_index(drop=True)[0]
        area=dens.loc[dens['variable']=='Area','value'].reset_index(drop=True)[0]
    else:
        pop_dens=dens.loc[dens['index']=='density',0].values[0]
        pop_total=dens.loc[dens['index']=='population',0].values[0]
        area=dens.loc[dens['index']=='area',0].values[0]   

    pop_dens_city.append(pop_dens)
    pop_city.append(pop_total)
    area_city.append(area)

    # pop density local, used for weighting distance and transit access metrics
    if country=='Germany':
        pop_dens_local=pd.read_csv('../outputs/density_geounits/' + city + '_pop_density.csv')
        pop_dens_local['Population']=pop_dens_local['Pop_2018']
    elif (country in ['France', 'Spain']) & (city != 'Paris'):
        pop_dens_local=pd.read_csv('../outputs/density_geounits/' + city + '_pop_density_mixres.csv')
        #pop_dens_local['Population']=pop_dens_local['area']*pop_dens_local['Density']
    elif city =='Paris':
        pop_dens_local=pd.read_csv('../outputs/density_geounits/' + city + '_pop_density_lowres.csv')
        pop_dens_local.rename(columns={'geo_unit':'geocode'},inplace=True)
    else:
        pop_dens_local=pd.read_csv('../outputs/density_geounits/' + city + '_pop_density.csv')

    # distances, weighted avg by population
    #data=pd.read_csv('../outputs/Combined/'+city+'_UF.csv')
    #data=data.loc[:,['Res_geocode','HHNR','DistSubcenter_res', 'DistCenter_res']].drop_duplicates()

    #data=pd.merge(data,pop_dens_local.loc[:,['geocode','Population']],left_on='Res_geocode',right_on='geocode')
    d2=pd.read_csv('../outputs/CenterSubcenter/' + city + '_dist.csv')
    data=pd.merge(d2,pop_dens_local.loc[:,['geocode','Population']],left_on='geocode',right_on='geocode')
    data=data.fillna(value=0.0) 
    # dc=np.average(data['DistCenter_res'],weights=data['Population'])
    # dsc=np.average(data['DistSubcenter_res'],weights=data['Population'])
    dc=np.average(data['Distance2Center'],weights=data['Population'])
    dsc=np.average(data['minDist_subcenter'],weights=data['Population'])
		
    d2c.append(dc)
    d2sc.append(dsc)

    # transit access
    fp='../outputs/transit_access/'+city+'.csv'
    tr=pd.read_csv(fp)
    # trmean=tr['score_spatiotemporal'].mean()
    tr_weighted=pd.merge(tr,pop_dens_local.loc[:,['geocode','Population']])
    tr_weighted=tr_weighted.fillna(value=0.0) 
    trmean=np.average(tr_weighted['score_spatiotemporal_min'], weights=tr_weighted['Population'])
    transit_access.append(trmean)



In [8]:
c=pd.Series(cities)
c.apply(get_UF_avg_lite)

Berlin
Dresden
Düsseldorf
Frankfurt am Main
Kassel
Leipzig
Magdeburg
Potsdam
Clermont
Dijon
Lille
Lyon
Montpellier
Nantes
Nimes
Paris
Toulouse
Madrid
Wien


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object

In [ ]:
c=pd.Series(cities)
c.apply(get_UF_avg)

Berlin


C:\Users\peter\AppData\Local\Temp\ipykernel_7232\1993487070.py:61: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../outputs/Combined/'+city+'_UF.csv')


Dresden
Düsseldorf
Frankfurt am Main
Kassel


C:\Users\peter\AppData\Local\Temp\ipykernel_7232\1993487070.py:61: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../outputs/Combined/'+city+'_UF.csv')


Leipzig
Magdeburg
Potsdam
Clermont
Dijon
Lille
Lyon
Montpellier
Nantes
Nimes
Paris
Toulouse
Madrid


C:\Users\peter\AppData\Local\Temp\ipykernel_7232\1993487070.py:61: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../outputs/Combined/'+city+'_UF.csv')


Wien


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object

In [9]:
df_summ_lite=pd.DataFrame({'cities':c,'Population':pop_city,'Area':area_city,'Pop. density':pop_dens_city,'Dist. to center':d2c,'Dist. to subcenter':d2sc,'Transit accessibility':transit_access})
df_summ_lite['Pop']=df_summ_lite['Population']*1e-6
df_summ_lite

,cities,Population,Area,Pop. density,Dist. to center,Dist. to subcenter,Transit accessibility,Pop
0,Berlin,3.748148e+06,891.177023,4205.840032,8.426453,2.362838,42.134284,3.748148
1,Dresden,5.654610e+05,328.261060,1722.595424,4.912877,3.023110,36.464499,0.565461
2,Düsseldorf,6.422470e+05,215.248167,2983.751307,4.461267,2.476586,31.511716,0.642247
3,Frankfurt am Main,7.473180e+05,251.287907,2973.951304,4.954017,2.074549,22.165990,0.747318
4,Kassel,2.049270e+05,108.101396,1895.692446,2.896925,2.593059,13.922025,0.204927
5,Leipzig,5.957760e+05,297.885248,2000.018480,4.439803,2.613412,20.009401,0.595776
6,Magdeburg,2.424780e+05,201.683212,1202.271608,3.326607,1.686011,19.810368,0.242478
7,Potsdam,1.781850e+05,188.339722,946.082952,3.880201,2.117823,8.486138,0.178185
8,Clermont,2.844050e+05,311.166203,913.997076,4.091220,2.382071,4.998102,0.284405
9,Dijon,2.521200e+05,241.068735,1045.842798,3.120561,2.606100,21.114873,0.252120


In [10]:
df_summ_lite.to_csv('../outputs/summary_stats/summary_UF_all_reweighted.csv',index=False)

In [ ]:
df_summ=pd.DataFrame({'cities':c,'Population':pop_city,'Area':area_city,'Pop. density':pop_dens_city,'Built-up Density':bld_dens_city,'Dist. to center':d2c,'Dist. to subcenter':d2sc,
                      'Intersec. Density':int_dens,'Street length avg.':street_len,'Cycle lane share':bike_share,'Urban fabric area':urb_fab,'Comm. area':comm,'Transit accessibility':transit_access})
df_summ.loc[:,['Cycle lane share','Urban fabric area','Comm. area']]=df_summ.loc[:,['Cycle lane share','Urban fabric area','Comm. area']]*100
df_summ['Pop']=df_summ['Population']*1e-6
df_summ

,cities,Population,Area,Pop. density,Built-up Density,Dist. to center,Dist. to subcenter,Intersec. Density,Street length avg.,Cycle lane share,Urban fabric area,Comm. area,Transit accessibility,Pop
0,Berlin,3.748148e+06,891.177023,4205.840032,1.099656,8.501587,2.365113,23.649297,144.011945,5.17,25.237160,10.365251,2958.281595,3.748148
1,Dresden,5.654610e+05,328.261060,1722.595424,0.544454,4.417224,2.948257,18.055751,140.135237,8.41,14.411046,7.430500,2406.562064,0.565461
2,Düsseldorf,6.422470e+05,215.248167,2983.751307,1.071483,4.599795,2.555426,27.558887,117.115021,8.82,19.800080,13.654648,2133.974869,0.642247
3,Frankfurt am Main,7.473180e+05,251.287907,2973.951304,0.299093,5.217463,2.187848,27.677779,121.508015,17.06,12.253403,12.084123,1439.983319,0.747318
4,Kassel,2.049270e+05,108.101396,1895.692446,0.138494,3.085375,2.655072,25.577838,124.848445,4.50,18.994395,9.457040,931.920679,0.204927
5,Leipzig,5.957760e+05,297.885248,2000.018480,0.739578,4.254474,2.389433,20.974520,130.186228,8.23,13.378511,12.289332,1414.379025,0.595776
6,Magdeburg,2.424780e+05,201.683212,1202.271608,0.627584,2.734132,1.798200,15.593762,144.723497,13.26,9.740381,8.351608,1275.409414,0.242478
7,Potsdam,1.782530e+05,188.339722,946.444002,0.259538,4.147738,2.007089,10.613799,147.280474,4.74,4.993357,2.677160,572.038101,0.178253
8,Clermont,2.844050e+05,311.166203,913.997076,0.449788,2.389599,1.815130,25.336974,97.337483,3.00,12.089983,7.702762,1173.618374,0.284405
9,Dijon,2.521200e+05,241.068735,1045.842798,0.404685,2.248755,2.536031,21.773911,105.650621,2.53,8.380765,7.753169,1355.590945,0.252120


In [ ]:
df_summ=pd.DataFrame({'cities':c,'Population':pop_city,'Area':area_city,'Pop. density':pop_dens_city,'Built-up Density':bld_dens_city,'Dist. to center':d2c,'Dist. to subcenter':d2sc,
                      'Intersec. Density':int_dens,'Street length avg.':street_len,'Cycle lane share':bike_share,'Urban fabric area':urb_fab,'Comm. area':comm,'Transit accessibility':transit_access})
df_summ.loc[:,['Cycle lane share','Urban fabric area','Comm. area']]=df_summ.loc[:,['Cycle lane share','Urban fabric area','Comm. area']]*100
df_summ['Pop']=df_summ['Population']*1e-6
df_summ

,cities,Population,Area,Pop. density,Built-up Density,Dist. to center,Dist. to subcenter,Intersec. Density,Street length avg.,Cycle lane share,Urban fabric area,Comm. area,Transit accessibility,Pop
0,Berlin,3.748148e+06,891.177023,4205.840032,1.099656,8.501587,2.365113,23.649297,144.011945,5.17,25.237160,10.365251,2958.281595,3.748148
1,Dresden,5.654610e+05,328.261060,1722.595424,0.544454,4.417224,2.948257,18.055751,140.135237,8.41,14.411046,7.430500,2406.562064,0.565461
2,Düsseldorf,6.422470e+05,215.248167,2983.751307,1.071483,4.599795,2.555426,27.558887,117.115021,8.82,19.800080,13.654648,2133.974869,0.642247
3,Frankfurt am Main,7.473180e+05,251.287907,2973.951304,0.299093,5.217463,2.187848,27.677779,121.508015,17.06,12.253403,12.084123,1439.983319,0.747318
4,Kassel,2.049270e+05,108.101396,1895.692446,0.138494,3.085375,2.655072,25.577838,124.848445,4.50,18.994395,9.457040,931.920679,0.204927
5,Leipzig,5.957760e+05,297.885248,2000.018480,0.739578,4.254474,2.389433,20.974520,130.186228,8.23,13.378511,12.289332,1414.379025,0.595776
6,Magdeburg,2.424780e+05,201.683212,1202.271608,0.627584,2.734132,1.798200,15.593762,144.723497,13.26,9.740381,8.351608,1275.409414,0.242478
7,Potsdam,1.782530e+05,188.339722,946.444002,0.259538,4.147738,2.007089,10.613799,147.280474,4.74,4.993357,2.677160,572.038101,0.178253
8,Clermont,2.848300e+05,311.166203,915.362906,0.449788,2.491963,1.851700,25.336974,97.337483,3.00,12.089983,7.702762,1125.258219,0.284830
9,Dijon,2.522080e+05,241.068735,1046.207839,0.404685,2.285327,2.541356,21.773911,105.650621,2.53,8.380765,7.753169,1333.328757,0.252208


In [ ]:
fp='../outputs/summary_stats/summary_UF_all.csv'
df_summ.to_csv(fp,index=False)

In [ ]:
df_summ=pd.read_csv('../outputs/summary_stats/summary_UF_all.csv')
df_summ

,cities,Population,Area,Pop. density,Built-up Density,Dist. to center,Dist. to subcenter,Intersec. Density,Street length avg.,Cycle lane share,Urban fabric area,Comm. area,Transit accessibility,Pop
0,Berlin,3.748148e+06,891.177023,4205.840032,1.099656,8.501587,2.365113,23.649297,144.011945,5.17,25.237160,10.365251,2958.281595,3.748148
1,Dresden,5.654610e+05,328.261060,1722.595424,0.544454,4.417224,2.948257,18.055751,140.135237,8.41,14.411046,7.430500,2406.562064,0.565461
2,Düsseldorf,6.422470e+05,215.248167,2983.751307,1.071483,4.599795,2.555426,27.558887,117.115021,8.82,19.800080,13.654648,2133.974869,0.642247
3,Frankfurt am Main,7.473180e+05,251.287907,2973.951304,0.299093,5.217463,2.187848,27.677779,121.508015,17.06,12.253403,12.084123,1439.983319,0.747318
4,Kassel,2.049270e+05,108.101396,1895.692446,0.138494,3.085375,2.655072,25.577838,124.848445,4.50,18.994395,9.457040,931.920679,0.204927
5,Leipzig,5.957760e+05,297.885248,2000.018480,0.739578,4.254474,2.389433,20.974520,130.186228,8.23,13.378511,12.289332,1414.379025,0.595776
6,Magdeburg,2.424780e+05,201.683212,1202.271608,0.627584,2.734132,1.798200,15.593762,144.723497,13.26,9.740381,8.351608,1275.409414,0.242478
7,Potsdam,1.782530e+05,188.339722,946.444002,0.259538,4.147738,2.007089,10.613799,147.280474,4.74,4.993357,2.677160,572.038101,0.178253
8,Clermont,2.844050e+05,311.166203,913.997076,0.449788,2.389599,1.815130,25.336974,97.337483,3.00,12.089983,7.702762,1173.618374,0.284405
9,Dijon,2.521200e+05,241.068735,1045.842798,0.404685,2.248755,2.536031,21.773911,105.650621,2.53,8.380765,7.753169,1355.590945,0.252120


In [ ]:
df_urb=pd.DataFrame({'cities':c,'Urban area':urb})
df_summ=df_summ.merge(df_urb)
df_summ['UrbPop Density']=df_summ['Pop. density']/df_summ['Urban area']
df_summ

,cities,Population,Area,Pop. density,Built-up Density,Dist. to center,Dist. to subcenter,Intersec. Density,Street length avg.,Cycle lane share,Urban fabric area,Comm. area,Transit accessibility,Pop,Urban area,UrbPop Density
0,Berlin,3.748148e+06,891.177023,4205.840032,1.099656,8.501587,2.365113,23.649297,144.011945,5.17,25.237160,10.365251,2958.281595,3.748148,0.684440,6144.935484
1,Dresden,5.654610e+05,328.261060,1722.595424,0.544454,4.417224,2.948257,18.055751,140.135237,8.41,14.411046,7.430500,2406.562064,0.565461,0.438014,3932.743961
2,Düsseldorf,6.422470e+05,215.248167,2983.751307,1.071483,4.599795,2.555426,27.558887,117.115021,8.82,19.800080,13.654648,2133.974869,0.642247,0.588414,5070.840952
3,Frankfurt am Main,7.473180e+05,251.287907,2973.951304,0.299093,5.217463,2.187848,27.677779,121.508015,17.06,12.253403,12.084123,1439.983319,0.747318,0.592050,5023.143001
4,Kassel,2.049270e+05,108.101396,1895.692446,0.138494,3.085375,2.655072,25.577838,124.848445,4.50,18.994395,9.457040,931.920679,0.204927,0.581253,3261.388013
5,Leipzig,5.957760e+05,297.885248,2000.018480,0.739578,4.254474,2.389433,20.974520,130.186228,8.23,13.378511,12.289332,1414.379025,0.595776,0.549292,3641.082403
6,Magdeburg,2.424780e+05,201.683212,1202.271608,0.627584,2.734132,1.798200,15.593762,144.723497,13.26,9.740381,8.351608,1275.409414,0.242478,0.427579,2811.809478
7,Potsdam,1.782530e+05,188.339722,946.444002,0.259538,4.147738,2.007089,10.613799,147.280474,4.74,4.993357,2.677160,572.038101,0.178253,0.315195,3002.726661
8,Clermont,2.844050e+05,311.166203,913.997076,0.449788,2.389599,1.815130,25.336974,97.337483,3.00,12.089983,7.702762,1173.618374,0.284405,0.371457,2460.571707
9,Dijon,2.521200e+05,241.068735,1045.842798,0.404685,2.248755,2.536031,21.773911,105.650621,2.53,8.380765,7.753169,1355.590945,0.252120,0.371388,2816.040390
